In [1]:
import sys
assert sys.version_info >= (3, 5)

import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

import numpy as np
import time

In [2]:
import os

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
os.environ["CUDA_VISIBLE_DEVICES"]="1"

#check os.environ ld_library_path is the same here as when I do it in python via terminal, if I get issues

#sometimes I can't select the GPU. In this case, try: https://forums.fast.ai/t/tip-limiting-tensorflow-to-one-gpu/1995

In [3]:
K = keras.backend


mnist = keras.datasets.mnist
(X_train_full, y_train_full), (X_test,y_test) = mnist.load_data()

X_valid, X_train = X_train_full[:5000] / 255.0, X_train_full[5000:] / 255.0

y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

X_test = X_test / 255.0

#1000 labelled points in train set for this example

X_train_la = X_train[:1000]
X_train_un = X_train[1000:]

y_train_la = y_train[:1000]
y_train_un = np.empty((X_train_la.shape[0]))


y_train_la = y_train_la.reshape(-1,1)

from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder()
y_train_la = encoder.fit_transform(y_train_la)

y_train_la=y_train_la.toarray()

y_valid = y_valid.reshape(-1,1)
y_valid = encoder.transform(y_valid)
y_valid = y_valid.toarray()

# M1 # 

In [5]:
#encoder

tf.random.set_seed(42)
np.random.seed(42)

class Sampling_M1(keras.layers.Layer):
    def call(self, inputs):
        mean, log_var = inputs
        return K.random_normal(tf.shape(log_var)) * K.exp(log_var/2) + mean

codings_size_M1 = 50

inputs_M1 = keras.layers.Input(shape=[28, 28])
z_M1 = keras.layers.Flatten()(inputs_M1)
z_M1 = keras.layers.Dense(600, activation="softplus")(z_M1)
z_M1 = keras.layers.Dense(300, activation="softplus")(z_M1)
codings_mean_M1 = keras.layers.Dense(codings_size_M1)(z_M1)
codings_log_var_M1 = keras.layers.Dense(codings_size_M1)(z_M1)
codings_M1 = Sampling_M1()([codings_mean_M1, codings_log_var_M1])
variational_encoder_M1 = keras.models.Model(
    inputs=[inputs_M1], outputs=[codings_mean_M1, codings_log_var_M1, codings_M1])


#decoder
decoder_inputs_M1 = keras.layers.Input(shape=[codings_size_M1])
x_M1 = keras.layers.Dense(300, activation="softplus")(decoder_inputs_M1)
x_M1 = keras.layers.Dense(600, activation="softplus")(x_M1)
x_M1= keras.layers.Dense(28 * 28, activation="sigmoid")(x_M1)
outputs_M1 = keras.layers.Reshape([28, 28])(x_M1)
variational_decoder_M1 = keras.models.Model(inputs=[decoder_inputs_M1], outputs=[outputs_M1])

#vae
_, _, codings_M1 = variational_encoder_M1(inputs_M1)
reconstructions_M1 = variational_decoder_M1(codings_M1)
variational_ae_M1 = keras.models.Model(inputs=[inputs_M1], outputs=[reconstructions_M1])

latent_loss_M1 = -0.5 * K.sum(
    1 + codings_log_var_M1 - K.exp(codings_log_var_M1) - K.square(codings_mean_M1),
    axis=-1)
variational_ae_M1.add_loss(K.mean(latent_loss_M1) / 784)
#question on how loss is computed too..
variational_ae_M1.compile(loss="binary_crossentropy", optimizer="rmsprop")


early_stopping_cb = keras.callbacks.EarlyStopping(patience=10,restore_best_weights=True)



In [60]:
history = variational_ae_M1.fit(X_train, X_train, epochs=1, batch_size=32,
                             validation_data=(X_valid, X_valid),  callbacks=[early_stopping_cb])

Train on 55000 samples, validate on 5000 samples
55000/55000 [==============================] - 15s 276us/sample - loss: 0.2089 - val_loss: 0.1731


### Attempt to save the model ###

In [61]:
config = variational_ae_M1.get_config()

In [62]:
custom_objects = {"Sampling_M1":Sampling_M1}

In [63]:
variational_ae_M1.save("M1_model.h5",save_format="tf")

In [67]:
model_Vae = keras.models.load_model("M1_model.h5",custom_objects=custom_objects)

In [47]:
variational_ae.save("M1_model.h5",save_format="tf") #doesn't work properly as need to deal with the sampling layer.
#refer to geron textbook for how to do this

In [14]:
#Generate the latent representations


_,_,X_train_la_encoded = variational_encoder(X_train_la)
X_train_la_encoded = X_train_la_encoded.numpy()

_,_,X_train_un_encoded = variational_encoder(X_train_un)
X_train_un_encoded = X_train_un_encoded.numpy()

_,_,X_valid_encoded = variational_encoder(X_valid)
X_valid_encoded = X_valid_encoded.numpy()

_,_,X_test_encoded = variational_encoder(X_test)
X_test_encoded = X_test_encoded.numpy()

In [45]:
np.save("X_train_la_encoded.npy",X_train_la_encoded)
np.save("X_train_un_encoded.npy",X_train_un_encoded)
np.save("X_valid_encoded.npy",X_valid_encoded)
np.save("X_test_encoded.npy",X_test_encoded)

In [26]:
X_train_un_encoded = np.load("X_train_un_encoded.npy")
X_train_la_encoded = np.load("X_train_la_encoded.npy")
X_valid_encoded = np.load("X_valid_encoded.npy")
X_test_encoded = np.load("X_test_encoded.npy")

# M2 # 

## Encoder ##

In [42]:
#need to set input and output shapes depending on x and y shapes
x_input_shape = 50

tf.random.set_seed(42)
np.random.seed(42)

#reparameterization trick
class Sampling(keras.layers.Layer):
    def call(self, inputs):
        mean, log_var = inputs
        return K.random_normal(tf.shape(log_var)) * K.exp(log_var/2) + mean

codings_size = 30

x_in = keras.layers.Input(shape=[50])
f = keras.layers.Flatten()(x_in)
z = keras.layers.Dense(40, activation="softplus")(f)
#z = keras.layers.Dense(300, activation="softplus")(z)

codings_mean = keras.layers.Dense(codings_size)(z)
codings_log_var = keras.layers.Dense(codings_size)(z)
codings = Sampling()([codings_mean, codings_log_var])

 

variational_encoder = keras.models.Model(
    inputs=[x_in], outputs=[codings_mean, codings_log_var, codings])

## Classifier ##

In [43]:
y_classifier = keras.layers.Dense(30, activation="selu")(f)
#y_classifier = keras.layers.Dense(100, activation="selu")(y_classifier)
y_pred = keras.layers.Dense(10,activation="softmax")(y_classifier) 

classifier = keras.models.Model(
    inputs=[x_in], outputs=[y_pred])

## Decoder ##

In [44]:

latent = keras.layers.Input(shape=[codings_size])
y = keras.layers.Input(shape=[10])

l_merged = keras.layers.concatenate([latent,y])
x = keras.layers.Dense(40, activation="softplus")(l_merged)
#x = keras.layers.Dense(600, activation="softplus")(x)
x = keras.layers.Dense(50, activation="sigmoid")(x)
x_out = keras.layers.Reshape([50])(x)


variational_decoder = keras.models.Model(inputs=[latent,y], outputs=[x_out])

## Compile model ##

In [45]:
#labelled vae
_,_, codings = variational_encoder(x_in)
y_pred = classifier(x_in)
reconstructions = variational_decoder([codings,y])
label_vae = keras.models.Model(inputs=(x_in,y), outputs=(reconstructions,y_pred))

In [46]:
#unlabelled vae
_,_, codings = variational_encoder(x_in)
y_pred = classifier(x_in)
reconstructions_un = variational_decoder([codings,y_pred])
unlabel_vae = keras.models.Model(inputs=x_in, outputs=reconstructions_un)

In [47]:
def labelled_loss_reconstruction(codings_log_var,codings_mean):
    def loss_functions_labelled(x, x_decoded_mean):
        x = K.reshape(x,[-1,50])
        x_decoded_mean = K.reshape(x_decoded_mean,[-1,50])
        xent_loss = 50*keras.losses.binary_crossentropy(x, x_decoded_mean)        
        kl_loss = - 0.5 * K.sum(1 + codings_log_var - K.square(codings_mean) - K.exp(codings_log_var), axis=-1)
        return K.mean(xent_loss + kl_loss)
    return loss_functions_labelled 

def unlabelled_loss_reconstruction(codings_log_var,codings_mean,y_pred):
    def loss_functions_unlabelled(x,x_decoded_mean):
        x = K.reshape(x,[-1,50])
        x_decoded_mean = K.reshape(x_decoded_mean,[-1,50])
        kl_loss = - 0.5 * K.sum(1 + codings_log_var - K.square(codings_mean) - K.exp(codings_log_var), axis=-1)
        xent_loss = 50*keras.losses.binary_crossentropy(x, x_decoded_mean)        
        entropy = keras.losses.categorical_crossentropy(y_pred,y_pred)
        loss = K.mean(kl_loss + xent_loss)
        #need to check below. We are summing over y, but we are assuming that the loss term is independent of y
        #which is not the case. How to do it though? https://github.com/bjlkeng/sandbox/issues/3
        #and how to do it for regression?
        return K.mean(K.sum(y_pred*loss,axis=-1)) + K.mean(entropy)
    return loss_functions_unlabelled

def labelled_cls_loss(y, y_pred,N=1000):
    alpha = 0.1*N
    cat_xent_loss = keras.losses.categorical_crossentropy(y, y_pred)
    return alpha*K.mean(cat_xent_loss)

In [48]:
label_vae.compile(loss=[labelled_loss_reconstruction(codings_log_var,codings_mean)
                        ,labelled_cls_loss], optimizer="rmsprop", experimental_run_tf_function=False)

unlabel_vae.compile(loss=unlabelled_loss_reconstruction(codings_log_var,codings_mean,y_pred),
                    optimizer="rmsprop", experimental_run_tf_function=False)

In [18]:
label_vae.summary()

Model: "model_8"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 50)]         0                                            
__________________________________________________________________________________________________
model_5 (Model)                 [(None, 50), (None,  241000      input_5[0][0]                    
__________________________________________________________________________________________________
input_7 (InputLayer)            [(None, 10)]         0                                            
__________________________________________________________________________________________________
model_7 (Model)                 (None, 50)           228950      model_5[1][2]                    
                                                                 input_7[0][0]              

In [19]:
#for debugging

history = label_vae.fit(
    [X_train_la_encoded,y_train_la], [X_train_la_encoded,y_train_la], epochs=1, #batch_size=128,
    validation_data=([X_valid_encoded,y_valid], [X_valid_encoded,y_valid])
)


Train on 1000 samples, validate on 5000 samples
1000/1000 [==============================] - 1s 1ms/sample - loss: 136.5636 - model_7_loss: 0.6303 - model_6_loss: 133.0965 - val_loss: 81.4951 - val_model_7_loss: -1.4265 - val_model_6_loss: 82.5993


In [ ]:

#for debugging

history = unlabel_vae.fit(
    [X_train_un_encoded], [X_train_un_encoded], epochs=1, #batch_size=128,
    validation_data=([X_valid_encoded], [X_valid_encoded])
)

## Create batches ##

In [49]:
#omer method - just picks things at random so each epoch may not necessarily go through every single point.
#but easier implementation for now

def create_batch(x_label, y_label, x_unlabel, batch_s=64):
    '''
    Creates batches of labelled and unlabelled data. The total number of points in both batches is equal to batch_s. 
    
    '''
    proportion_labelled = x_label.shape[0]/(x_label.shape[0] + x_unlabel.shape[0])
    
    shape_label = x_label.shape[0]
    label_per_batch = int(np.ceil(proportion_labelled*batch_s))
    batch_idx_la = np.random.choice(list(range(shape_label)), label_per_batch)
    batch_x_la = (x_label[batch_idx_la, :])
    batch_y_la = (y_label[batch_idx_la])
    
    shape_unlabel = x_unlabel.shape[0]
    unlabel_per_batch = batch_s - label_per_batch
    batch_idx_un = np.random.choice(list(range(shape_unlabel)), unlabel_per_batch)
    batch_x_un = (x_unlabel[batch_idx_un, :])
    
    del batch_idx_la,batch_idx_un
            
    return batch_x_la, batch_y_la, batch_x_un

## Train ##

In [50]:
def progress_bar(iteration, total, size=30):
    running = iteration < total
    c = ">" if running else "="
    p = (size - 1) * iteration // total
    fmt = "{{:-{}d}}/{{}} [{{}}]".format(len(str(total)))
    params = [iteration, total, "=" * p + c + "." * (size - p - 1)]
    return fmt.format(*params)

def print_status_bar(iteration, total, loss, metrics=None, size=30):
    metrics = " - ".join(["Loss: {:.4f}".format(loss)])
    end = "" if iteration < total else "\n"
    print("\r{} - {}".format(progress_bar(iteration, total), metrics), end=end)
    
def print_status_bar_epoch(iteration, total, loss, validation_loss, metrics=None, size=30):
    metrics = " - ".join(["Loss: {:.4f} Validation loss: {:.4f} ".format(loss,validation_loss)])
    end = "" if iteration < total else "\n"
    print("\r{} - {}".format(progress_bar(iteration, total), metrics), end=end)
    
#could make these functions into just one which works for both if I want. See the Geron textbook.

In [51]:
def fit_model(X_train_la, y_train_la, X_train_un,epochs,X_valid_la, y_valid_la,patience,batch_size=64):

    """
    Fits the model. Gets the validation loss too. And includes early stopping, given by the patience.
    
    """
    #The callback still doesn't take the model back to the model which performed best.. how to implement that?
    #maybe save the model after each epoch in a list of max size 10.
    #if we callback is reached, look at val loss list and see index which is smallest and choose that for model restoration
    
    start = time.time()
    history = []
    
    validation_loss = []
    
    batches_per_epoch = int(np.floor((X_train_la.shape[0] + X_train_un.shape[0])/batch_size))

    for epoch in range(epochs):
            
            print("Epoch {}/{}".format(epoch,epochs))
            
            for i in range(batches_per_epoch):

                batch_x_la, batch_y_la, batch_x_un = create_batch(X_train_la,y_train_la,X_train_un,batch_size)

                loss = unlabel_vae.train_on_batch(batch_x_un,batch_x_un)

                loss += label_vae.train_on_batch([batch_x_la,batch_y_la],
                                                [batch_x_la,batch_y_la])[0] #selecting the overall loss term
                

                history.append(loss)
                print_status_bar(i*batch_size,X_train_la.shape[0] + X_train_un.shape[0],loss)
                
            val_loss = unlabel_vae.evaluate(X_train_un,X_train_un,verbose=0);
            
            val_loss += label_vae.evaluate([X_train_la,y_train_la],[X_train_la,y_train_la],verbose=0)[0];
            
            validation_loss.append(val_loss)
                
            print_status_bar_epoch(X_train_la.shape[0] + X_train_un.shape[0]
                             ,(X_train_la.shape[0] + X_train_un.shape[0]),loss,val_loss)
            
            #callback for early stopping
            if epoch > patience - 1:
                
                latest_val_loss = validation_loss[-patience:]
                if all(i<=val_loss for i in latest_val_loss) is True:
                    break
            
            
                
    done = time.time()
    elapsed = done-start
    print("Elapsed: ",elapsed)
    print("Final training loss: ",loss)
    
    return history

In [52]:
history = fit_model(X_train_la_encoded, y_train_la, X_train_un_encoded,10,X_valid_encoded,y_valid,patience=10,batch_size=64)

Epoch 0/10
55000/55000 [==============================] - Loss: -34.9923 Validation loss: 257.3728 
Epoch 1/10
55000/55000 [==============================] - Loss: -149.3890 Validation loss: 41.8777 
Epoch 2/10
55000/55000 [==============================] - Loss: -317.7199 Validation loss: -87.4285 
Epoch 3/10
55000/55000 [==============================] - Loss: -372.6899 Validation loss: -135.5260 
Epoch 4/10
55000/55000 [==============================] - Loss: -152.3757 Validation loss: -181.5146 
Epoch 5/10
55000/55000 [==============================] - Loss: -189.3367 Validation loss: -207.4836 
Epoch 6/10
55000/55000 [==============================] - Loss: -334.2572 Validation loss: -245.6570 
Epoch 7/10
55000/55000 [==============================] - Loss: -251.5693 Validation loss: -275.2056 
Epoch 8/10
55000/55000 [==============================] - Loss: -248.8307 Validation loss: -288.2636 
Epoch 9/10
55000/55000 [==============================] - Loss: -308.4933 Validation lo

# Test #

In [39]:
X_test_encoded.shape

(10000, 50)

In [29]:
y_test = y_test.reshape(-1,1)
y_test = encoder.transform(y_test)
y_test = y_test.toarray()

In [30]:
y_test.shape

(10000, 10)

In [53]:
y_prediction = classifier.predict(X_test_encoded)

In [54]:
from sklearn.metrics import log_loss

xent_loss = log_loss(y_test,y_prediction)

In [55]:
xent_loss

1.443769420192088

In [56]:
y_pred_rounded = np.round(y_prediction,0)

In [57]:
from sklearn.metrics import accuracy_score as accuracy

accuracy_score = accuracy(y_test,y_pred_rounded)

accuracy_score

0.4218

Results are very poor. I could have run it for longer. When I did run a different set up for longer, accuracy was just 0.6. Maybe because of how I configured the M2 model. M1 gives it a sample with 50 features, and then M2's coding layer is also 50 features. We want to decrease the latent dimensionality. Also the number of nodes are maybe too big. 

In the run shown, I've configured it differently, but still poor results. 

Maybe the models need to be trained jointly, as in having a probabilistic model overall, unlike what is done here, where M1 is used as a feature extractor? Can try this out...

In [59]:
classifier.save("m1+m2_poor_classifier.h5") 